In [34]:
import math
import numpy as np 

In [35]:
sqrt, cos, sin, pi = math.sqrt, math.cos, math.sin, math.pi
"前处理"
nodeNumber = 4
KK = np.zeros((2*nodeNumber, 2*nodeNumber))

In [36]:
print(KK)

[[0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]]


In [37]:
U = np.array([0, 0, 'u_Unknown', 0, 'u_Unknown', 'u_Unknown', 0, 0], dtype=object)
F = np.array(['f_Unknown', 'f_Unknown', 2e4, 'f_Unknown', 0, -2.5e4, 'f_Unknown', 'f_Unknown'], dtype=object)

In [38]:
class Bar2D:
    """定义二维杆单元类，该类包含杆件的基本信息：
    E 弹性模量，A 杆单元面积，i 单元起点的节点编号，j 单元终点的节点编号
    x1 y1 起点的坐标，x2 y2 终点的坐标，
    DOF 单元在总体刚度矩阵里面所在的位置,L 单元的长度，
    cos sin 单元的方向余弦 方向正弦，
    k 单元刚度矩阵"""
    def __init__(self, E, A, x1, y1, x2, y2, i, j):
        self.E = E
        self.A = A
        self.i = i
        self.j = j
        # 定义一个由单刚矩阵的自由度向总刚矩阵自由度转换的数组
        self.DOF = np.array([2*i-2, 2*i-1, 2*j-2, 2*j-1],dtype=np.int16)
        self.L = sqrt((x1 - x2)**2 + (y1 - y2)**2)
        self.cos = (x2 - x1) / self.L
        self.sin = (y2 - y1) / self.L
        L, c, s = self.L, self.cos, self.sin
        self.k = (E*A/L)*np.array([[c*c, c*s, -c*c, -c*s],
                                    [c*s, s*s, -c*s, -s*s],
                                    [-c*c, -c*s, c*c, c*s],
                                    [-c*s, -s*s, c*s, s*s]])
    "定义求解单元应力的函数"
    def stress(U):
        # 从位移矩阵U中获取该单元两个节点的1*4位移矩阵
        u = U(self.DOF)
        E, L, c, s = self.E, self.L, self.c, self.s
        T = np.array([-c, -s, c, s])
        self.bar_Stress = E / L * np.dot(T, u)
        return self.bar_Stress


In [39]:
def Bar2D2Node_Assembly(KK, bar):
    for  n1 in range(4):
        for n2 in range(4):
            KK[bar.DOF[n1], bar.DOF[n2]] += bar.k[n1, n2]
    return KK

In [40]:
def node_Disaplacement(KK, U, F):
    # 获取缩减后的总刚矩阵
    del_row_col = np.where(U == 0)
    kk_delRow = np.delete(KK, del_row_col, 0)
    kk_delCol = np.delete(kk_delRow, del_row_col,1)
    kk = kk_delCol
    # 获取节点位移位置对应的节点力矩阵
    f = F[np.where(U == 'u_Unknown')]
    f=np.array(f,dtype='float')
    u = np.linalg.solve(kk, f)
    U[np.where(U=='u_Unknown')] = u
    return U

In [41]:
'求解节点力，必须在已经求得节点位移U后才可调用本函数'
def node_Force(KK, U):
    F = np.dot(KK, U)
    return F

In [42]:
E, A, x1, y1, x2, y2, x3, y3, x4, y4 = 2.95e11, 0.0001, 0, 0, 0.4, 0, 0.4, 0.3, 0, 0.3
bar1 = Bar2D(E, A, x1, y1, x2, y2, 1, 2)
bar2 = Bar2D(E, A, x3, y3, x2, y2, 3, 2)
bar3 = Bar2D(E, A, x1, y1, x3, y3, 1, 3)
bar4 = Bar2D(E, A, x4, y4, x3, y3, 4, 3)

bars = [bar1, bar2, bar3, bar4]

for bar in bars:
    Bar2D2Node_Assembly(KK, bar)

In [43]:
# 求解位移
U = node_Disaplacement(KK, U, F)
# 求解节点力
F = node_Force(KK, U)

In [44]:
print(U)
print(F)

[0 0 0.0002711864406779661 0 5.649717514124294e-05 -0.00022245762711864406
 0 0]
[-15833.333333333336 3125.0 20000.0 21875.0 0.0 -25000.0
 -4166.666666666667 0.0]
